In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, date, time

## Cargar los datos

In [2]:
datos_electricos = pd.read_csv("data_proyecto.csv", encoding= "utf-8")
datos_electricos["energy(Wh)"] = datos_electricos["energy(Wh)"]/1000
datos_electricos["power(W)"] = datos_electricos["power(W)"]/1000
datos_electricos = datos_electricos.rename(columns = {"power(W)":"power(kW)","energy(Wh)":"energy(kWh)"})

datos_electricos.head()

,device_id,device_name,measurement_time(UTC),current(A),voltage(V),power(kW),power_factor,energy(kWh)
0,82608,Linea 2 Tablero Principal,1/1/2019 0:00,0.71,135.3,0.0931,0.97,0.0233
1,73170,L3 Transportador Botellas PDC III,1/1/2019 0:00,2.94,127.0,0.3551,0.95,0.0888
2,72819,Tablero C Alumbrado Linea Produccion,1/1/2019 0:00,6.34,127.0,0.7644,0.95,0.1911
3,72823,L4 Control de Inyectores,1/1/2019 0:00,30.97,127.0,3.7360,0.95,0.9340
4,82649,L4 Mezcladora Paramix,1/1/2019 0:00,4.79,127.0,0.5780,0.95,0.1445


In [3]:
datos_grupos = pd.read_excel("Device_Groups.xlsx")
datos_grupos.head()

,device_name,DG1,DG2,Usable
0,Alumbrado General L1 y Tratamiento Aguas,Potencia Industrial,Otros,0
1,Compresor 1 Aire 250 HP,Potencia Industrial,Compresores de aire,1
2,Compresor 2 Aire 200 HP,Potencia Industrial,Compresores de aire,1
3,Compresor de Aire 1 60 HP,Potencia Industrial,Compresores de aire,1
4,Compresor de Aire 2 60 HP,Potencia Industrial,Compresores de aire,1


In [4]:
datos_produccion = pd.read_excel("produccion_lineas.xlsx")
fecha_formato = [datetime.strptime(date,"%d/%m/%Y") for date in datos_produccion["Fecha"].values.tolist()]
datos_produccion["Fecha"] = fecha_formato
datos_produccion["Fecha"] = datos_produccion["Fecha"].dt.date
datos_produccion.head()

,Fecha,Prod. Linea 1(LTS),Prod. Linea 2(LTS),Prod. Linea 3(LTS),Prod. Linea 4(LTS)
0,2019-07-31,0,0.0,0,0
1,2019-07-30,0,0.0,0,0
2,2019-07-29,0,0.0,0,0
3,2019-07-28,0,0.0,0,0
4,2019-07-27,0,0.0,0,0


In [5]:
datos_electricos2 = datos_electricos.drop(columns=["device_id","current(A)","voltage(V)","power_factor","energy(kWh)"])
datos_electricos2.head()

,device_name,measurement_time(UTC),power(kW)
0,Linea 2 Tablero Principal,1/1/2019 0:00,0.0931
1,L3 Transportador Botellas PDC III,1/1/2019 0:00,0.3551
2,Tablero C Alumbrado Linea Produccion,1/1/2019 0:00,0.7644
3,L4 Control de Inyectores,1/1/2019 0:00,3.7360
4,L4 Mezcladora Paramix,1/1/2019 0:00,0.5780


In [6]:
datos_electricos3=pd.merge(left = datos_electricos2, right = datos_grupos, on="device_name", how = "left").drop(columns="DG1")
datos_electricos3.head()

,device_name,measurement_time(UTC),power(kW),DG2,Usable
0,Linea 2 Tablero Principal,1/1/2019 0:00,0.0931,Linea 2 Total,1.0
1,L3 Transportador Botellas PDC III,1/1/2019 0:00,0.3551,Linea 3,0.0
2,Tablero C Alumbrado Linea Produccion,1/1/2019 0:00,0.7644,Otros,0.0
3,L4 Control de Inyectores,1/1/2019 0:00,3.7360,Linea 4 Total,1.0
4,L4 Mezcladora Paramix,1/1/2019 0:00,0.5780,NaN,NaN


In [7]:
datos_electricos_matriz = datos_electricos3[(datos_electricos3["Usable"]==1 )].drop(columns= "Usable")
datos_electricos_matriz.head()

,device_name,measurement_time(UTC),power(kW),DG2
0,Linea 2 Tablero Principal,1/1/2019 0:00,0.0931,Linea 2 Total
3,L4 Control de Inyectores,1/1/2019 0:00,3.7360,Linea 4 Total
15,L4 PDC II,1/1/2019 0:00,4.2513,Linea 4 Total
16,Compresor de Amoniaco 2 Tornillo 250 HP VFD,1/1/2019 0:00,0.4013,Refrigeración L1L3L4
17,Compresor de Aire 1 60 HP,1/1/2019 0:00,0.0072,Compresores de aire


### Obtener datos para regresiones

In [8]:
datos_electricos4 = datos_electricos.drop(columns=["device_id","current(A)","voltage(V)","power_factor","power(kW)"])
datos_electricos4 = pd.merge(left = datos_electricos4, right = datos_grupos, on="device_name", how = "left").drop(columns="DG1")
datos_electricos4 =datos_electricos4[datos_electricos4["Usable"]==1].drop(columns="Usable")
datos_electricos4.head()

,device_name,measurement_time(UTC),energy(kWh),DG2
0,Linea 2 Tablero Principal,1/1/2019 0:00,0.0233,Linea 2 Total
3,L4 Control de Inyectores,1/1/2019 0:00,0.9340,Linea 4 Total
15,L4 PDC II,1/1/2019 0:00,1.0628,Linea 4 Total
16,Compresor de Amoniaco 2 Tornillo 250 HP VFD,1/1/2019 0:00,0.1003,Refrigeración L1L3L4
17,Compresor de Aire 1 60 HP,1/1/2019 0:00,0.0018,Compresores de aire


In [9]:
fecha_formato = [datetime.strptime(date,"%d/%m/%Y %H:%M") for date in datos_electricos4["measurement_time(UTC)"].values.tolist()]
datos_electricos4["measurement_time(UTC)"] = fecha_formato

In [10]:
datos_electricos4 = datos_electricos4[datos_electricos4["measurement_time(UTC)"] < datetime(2019,7,1,0,0)]
datos_electricos4.head()

,device_name,measurement_time(UTC),energy(kWh),DG2
0,Linea 2 Tablero Principal,2019-01-01,0.0233,Linea 2 Total
3,L4 Control de Inyectores,2019-01-01,0.9340,Linea 4 Total
15,L4 PDC II,2019-01-01,1.0628,Linea 4 Total
16,Compresor de Amoniaco 2 Tornillo 250 HP VFD,2019-01-01,0.1003,Refrigeración L1L3L4
17,Compresor de Aire 1 60 HP,2019-01-01,0.0018,Compresores de aire


In [11]:
datos_electricos4["Fecha"] = datos_electricos4["measurement_time(UTC)"].dt.date
datos_electricos_regresion = pd.pivot_table(datos_electricos4.groupby(["Fecha","DG2"]).sum().reset_index(),
                                            index="Fecha", columns="DG2", values="energy(kWh)",
                                           aggfunc="sum").reset_index()
datos_electricos_regresion.head()

DG2,Fecha,Compresores de aire,Linea 1 Total,Linea 2 Total,Linea 3 Total,Linea 4 Total,Refrigeración L1L3L4,Refrigeración L2
0,2019-01-01,1214.4367,1281.9504,87.7999,167.8932,624.6109,1019.0041,0.4663
1,2019-01-02,4610.1620,4391.1099,883.5941,843.6383,3473.6623,5220.9873,263.6025
2,2019-01-03,4646.8085,3345.8780,1850.9111,2477.3533,2907.2174,4117.7892,2500.2312
3,2019-01-04,5128.9255,2959.1534,2102.0838,2815.7259,3600.7994,4275.8102,2967.9332
4,2019-01-05,4565.3341,3393.3607,465.6202,2394.8061,3003.6845,4567.4457,1240.4406


In [12]:
pd.merge(left=datos_electricos_regresion, right = datos_produccion, left_on="Fecha", right_on="Fecha")


,Fecha,Compresores de aire,Linea 1 Total,Linea 2 Total,Linea 3 Total,Linea 4 Total,Refrigeración L1L3L4,Refrigeración L2,Prod. Linea 1(LTS),Prod. Linea 2(LTS),Prod. Linea 3(LTS),Prod. Linea 4(LTS)
0,2019-01-01,1214.4367,1281.9504,87.7999,167.8932,624.6109,1019.0041,0.4663,640512,0.0,0,399168
1,2019-01-02,4610.1620,4391.1099,883.5941,843.6383,3473.6623,5220.9873,263.6025,820448,204508.8,326400,1150848
2,2019-01-03,4646.8085,3345.8780,1850.9111,2477.3533,2907.2174,4117.7892,2500.2312,1108224,690508.8,799360,1114560
3,2019-01-04,5128.9255,2959.1534,2102.0838,2815.7259,3600.7994,4275.8102,2967.9332,533504,685843.2,577920,995328
4,2019-01-05,4565.3341,3393.3607,465.6202,2394.8061,3003.6845,4567.4457,1240.4406,442880,65318.4,335360,644112
5,2019-01-06,1876.6186,1876.0512,139.1685,198.3137,806.4864,1448.0335,0.0000,916992,0.0,0,393984
6,2019-01-07,4522.6533,3670.3405,1466.0789,1679.4776,3308.1581,4930.1967,1855.0366,631680,713836.8,721920,1136592
7,2019-01-08,4450.8017,409.9689,1706.9276,2515.3073,3200.8524,971.6763,1927.2886,0,392688.0,162560,1101600
8,2019-01-09,3180.9719,2282.0332,28.8108,1087.5534,3075.5556,1577.5164,0.0010,600192,0.0,0,946080
9,2019-01-10,3690.3769,3637.7239,528.7063,2183.8896,3014.3048,4672.4053,497.1724,949248,309484.8,898560,1140480


## Limpieza de datos